In [ ]:
# manual nested cross-validation for random forest on a classification dataset
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def build_classifier(optimizer):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
  model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
  model.add(tf.keras.layers.Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

my_model = KerasClassifier(build_fn = build_classifier)

In [ ]:
# create dataset
import pandas as pd
dataset = pd.read_csv('Churn_Modelling.csv')  
X = dataset.iloc[:, 3: 13].values
y = dataset.iloc[:, 13].values

from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder() #instantiate an object of the class LabelEncoder
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #ordinal encoding for column 1

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) #ordinal encoding for column 2

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
ct = ColumnTransformer( #'encoder' is the name of the column transformer
    [('encoder', OneHotEncoder(), [1])],    # The column numbers to be transformed (here is [1] but can be [0, 1, 3])
    remainder='passthrough'                         # Leave the rest of the columns untouched
)
X = np.array(ct.fit_transform(X), dtype=np.float)

X = X[:, 1:]

#Standardise the data (x_standardised = (x - x_mean)/std_dev)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
'''X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) #note that we use the scale set from the training set to transform the test set'''
X = sc.fit_transform(X)

In [ ]:
# configure the outer cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
	X_train, X_test = X[train_ix, :], X[test_ix, :]
	y_train, y_test = y[train_ix], y[test_ix]
	# configure the inner cross-validation procedure
	cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)
	# define the model
	model = my_model
	# define search space : A dictionary containing the parameters with the values we want the model to try on and apply grid search
	space = {'batch_size': [20, 30], 'nb_epoch': [50, 100], 'optimizer': ['adam', 'sgd']}
	# define search
	search = GridSearchCV(model, space, scoring='accuracy', cv=cv_inner, refit=True)
	# execute search
	result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
	best_model = result.best_estimator_
	# evaluate model on the hold out dataset
	yhat = best_model.predict(X_test)
	# evaluate the model
	acc = accuracy_score(y_test, yhat)
	# store the result
	outer_results.append(acc)
	# report progress
	print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

405/405 [==============================] - 1s 1ms/step - loss: 0.6204 - accuracy: 0.7812


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


450/450 [==============================] - 1s 1ms/step - loss: 0.5920 - accuracy: 0.7985
>acc=0.789, est=0.798, cfg={'batch_size': 20, 'nb_epoch': 100, 'optimizer': 'adam'}
450/450 [==============================] - 1s 1ms/step - loss: 0.6042 - accuracy: 0.7974
>acc=0.796, est=0.796, cfg={'batch_size': 20, 'nb_epoch': 50, 'optimizer': 'adam'}
450/450 [==============================] - 1s 1ms/step - loss: 0.6092 - accuracy: 0.7883
>acc=0.788, est=0.797, cfg={'batch_size': 20, 'nb_epoch': 50, 'optimizer': 'adam'}
450/450 [==============================] - 1s 1ms/step - loss: 0.6141 - accuracy: 0.7998
>acc=0.786, est=0.797, cfg={'batch_size': 20, 'nb_epoch': 50, 'optimizer': 'adam'}
450/450 [==============================] - 1s 1ms/step - loss: 0.6258 - accuracy: 0.7874
>acc=0.815, est=0.795, cfg={'batch_size': 20, 'nb_epoch': 50, 'optimizer': 'adam'}
450/450 [==============================] - 1s 1ms/step - loss: 0.6052 - accuracy: 0.7944
>acc=0.796, est=0.796, cfg={'batch_size': 20, 'nb_